## Agents as Tools con Strands Agents

- Documentación: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agents-as-tools

#### Sub-agente 1: Asistente de Viajes

In [ ]:
from strands import Agent, tool
from strands_tools import retrieve, http_request

@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            system_prompt="""You are a specialized travel planning assistant.
            Do not use more than 1 source, as soon as you receive data from it.
            Create detailed travel itineraries based on user preferences.""",
            tools=[retrieve, http_request],  # Travel information tools
        )
        # Implementation with response handling
        response = travel_agent(query)
        
        return response
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

#### Sub-agente 2: Pronóstico Meteorológico

In [ ]:
@tool
def _weather_tool_api(location: str) -> str:
    """Get weather forecast for a location for the next 2 weeks.

    Args:
        location: City or location name
    """

    # Implement weather lookup logic here
    return f"Weather for {location}: Soleado, 34°C"

@tool
def weather_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            system_prompt="""You are a specialized weather forecasting assistant.""",
            tools=[_weather_tool_api],  # Travel information tools
        )
        # Implementation with response handling
        response = travel_agent(query)
        
        return response
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Creamos el Orquestador de Agentes

In [ ]:
# Define the orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are a travel assistant that routes queries to specialized agents:
- For weather forecasting → Use weather_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

# Strands Agents SDK allows easy integration of agent tools
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    callback_handler=None,
    tools=[trip_planning_assistant, weather_assistant]
)

#### Opcional: Podemos activar Logging

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [ ]:
# Example: E-commerce Customer Service System
customer_query = """
Estoy buscando botas para escalar, para un viaje que tengo la sig semana al Nevado del Tolima en Colombia.
Tomando en cuenta el clima, dame un itinerario para una pareja de 2 peronas en sus treintas de edad.
"""

# The orchestrator automatically determines that this requires multiple specialized agents
response = orchestrator(customer_query)